\<w\> : english word  
\<t\> : type of the word (noun, verb...)  
\<b\> : meaning - definition  
\<s\> : subtype of the word (masculine, plural...)  
\<i\> : correspondant amazigh word  
\<x\> : an example illustrating the use of this amazigh word  

In [ ]:
import sqlite3

In [220]:
# utils


class AmWord(object):
    def __init__(self, am_word, examples: list[str]):
        self.am_word  = am_word
        self.examples:list[str] = examples
    
    def __str__(self):
        res = f"{self.am_word}: {self.examples}"
        return res

    

class SubDefinition(object):
    def __init__(self, sub_type='', am_words=[]):
        self.sub_type = sub_type
        self.am_words = am_words
    
    def __str__(self):
        res = f"(sub_type: {self.sub_type}):\n"
        for am in self.am_words:
            res += '\t'
            res += am.__str__()
            res += ',\n'
        return res + ']'
    

class Definition(object):
    
    def __init__(self, definition='', sub_definitions=[]):
        self.definition      = definition 
        self.sub_definitions = sub_definitions
        
    
    def __str__(self):
        res = f"{self.definition}\n"
        for sub in self.sub_definitions:
            res += sub.__str__() + '\n'
        return res
        
        

class DefinitionByType(object):
    def __init__(self, word_type, definitions):
        self.word_type = word_type
        self.definitions = definitions
    
    
    def __str__(self):
        res = f"(type: {self.word_type}):\n"
        for defi in self.definitions:
            res += defi.__str__()
        return res
    
        
class EnWord(object):
    def __init__(self, en_word, definitions_by_type):
        self.en_word = en_word
        self.definitions_by_type = definitions_by_type
    
    def __str__(self):
        res = f"--{self.en_word}--\n"
        for defi in self.definitions_by_type:
            res += defi.__str__()
        return res + '\n'
        

"""============================================================================================="""
        
        
def get_examples(content):
    for i in range(len(content)):
        if not content[i].startswith('<x>'):
            raise Exception(f"not an example: {content[i]}")
        
        content[i] = content[i][3:].strip()
    return content
    
    
def get_am_words(content):
    i_indexes = []
    for i, line in enumerate(content):
        if line.startswith('<i>'):
            i_indexes.append(i)
    
    if len(i_indexes) == 0:
        return [AmWord('', get_examples(content))]
    
    am_words = []
    
    if i_indexes[0] != 0:
        am_words.append(AmWord('', get_examples(content[:i_indexes[0]])))
    
    con = None
    for i in range(len(i_indexes)):
        if i == len(i_indexes) - 1:
            con = content[i_indexes[i]+1:]
        else:
            con = content[i_indexes[i]+1:i_indexes[i+1]]
        am_words.append(AmWord(content[i_indexes[i]][3:].strip(), get_examples(con)))
        
    return am_words
        
def get_sub_definitions(content):
    s_indexes = []
    for i, line in enumerate(content):
        if line.startswith('<s>'):
            s_indexes.append(i)
    
    if len(s_indexes) == 0:
        return [SubDefinition('', get_am_words(content))]
    
    sub_definitions = []
    
    if s_indexes[0] != 0:
        sub_definitions.append(SubDefinition('', get_am_words(content[:s_indexes[0]])))
    
    con = None
    for i in range(len(s_indexes)):
        if i == len(s_indexes) - 1:
            con = content[s_indexes[i]+1:]
        else:
            con = content[s_indexes[i]+1:s_indexes[i+1]]
        sub_definitions.append(SubDefinition(content[s_indexes[i]][3:].strip(), get_am_words(con)))
        
    return sub_definitions
        

def get_definitions(content):
    b_indexes = []
    for i, line in enumerate(content):
        if line.startswith('<b>'):
            b_indexes.append(i)
    
    if len(b_indexes) == 0:
        return [Definition('', get_sub_definitions(content))]
    
    definitions = []
    
    if b_indexes[0] != 0:
        definitions.append(Definition('', get_sub_definitions(content[:b_indexes[0]])))
    
    con = None
    for i in range(len(b_indexes)):
        if i == len(b_indexes) - 1:
            con = content[b_indexes[i]+1:]
        else:
            con = content[b_indexes[i]+1:b_indexes[i+1]]
        definitions.append(Definition(content[b_indexes[i]][3:].strip(), get_sub_definitions(con)))
    
    return definitions
        
def get_definitions_by_type(content):
    t_indexes = []
    for i, line in enumerate(content):
        if line.startswith('<t>'):
            t_indexes.append(i)
            
    if len(t_indexes) == 0:
        return [DefinitionByType('', get_definitions(content))]
    
    definitions_by_type = []

    if t_indexes[0] != 0:
        definitions_by_type.append(DefinitionByType('', get_definitions(content[:t_indexes[0]])))
      
    con = None
    for i in range(len(t_indexes)):
        if i == len(t_indexes) - 1:
            con = content[t_indexes[i]+1:]
        else:
            con = content[t_indexes[i]+1: t_indexes[i+1]]
            
        definitions_by_type.append(DefinitionByType(content[t_indexes[i]][3:].strip(), get_definitions(con)))
        
    return definitions_by_type

    
    

        
def get_word(content):
    if not content[0].startswith('<w>'):
        raise Exception(f"couldn't find the english word {content[0]}")
        
    en_word = content[0][3:].strip()
    definitions_by_type = get_definitions_by_type(content[1:])
    
    return EnWord(en_word, definitions_by_type)

In [231]:
# testing
content = """<w>a
<t>(art.)
<s>mas.
<i>yan
<x>ẓeṛiɣ yan urgaz
<s>fem.
<i>yat
<x>yat tefruxt ad nn-unniɣ"""
content2 = """<w>zucchini squach
<t>(n.)
<i>taxsayt izbubn, tixsayin izbuben
<x>ur ar ictta taxsayt izbubn
<i>taḥṛcit, tiḥṛciyin
<x>ur ar ictta taḥṛcit"""
content = content2.split('\n')
word = get_word(content)
# print(word)
con = sqlite3.connect("db.sqlite3")
cursor = con.cursor()
save_word(word, cursor)
con.commit()
con.close()


In [214]:
t_ids = {'': 1, '(aux. v.)': 2, '(comp.)': 3, '(pron. & conj.)': 4, '(art.)': 5, '(pref.)': 6, '(adj. & adv. & pron.)': 7, '(interj.)': 8, '(conj. & adj. & pron.)': 9, '(prefix.)': 10, '(exp.)': 11, '(perp.)': 12, '(prep. & conj.)': 13, '(adj. & pron. & conj.)': 14, '(pl. n.)': 15, '(adj. & pron.)': 16, '(adv & conj.)': 17, '(adv. & pron.)': 18, '(n. & adj. & adv.)': 19, '(expr.)': 20, '(conj. & pron.)': 21, '(det.)': 22, '(adv. & n.)': 23, '(conj.)': 24, '(n. pl.)': 25, '(n.)': 26, '(v.)': 27, '(adj & n.)': 28, '(adv.)': 29, '(adj. & conj.)': 30, '(adj. & adv.)': 31, '(super.)': 32, '(pre. & adv.)': 33, '(adv & prep.)': 34, '(pron.)': 35, '(adj. & n.)': 36, '(adj. & prep.)': 37, '(adv. & prep.)': 38, '(prep. & conj. & adv.)': 39, '(adj.)': 40, '(prep.)': 41, '(f&m. use.)': 42, '(inter.)': 43, '(adv. & prep. & adj.)': 44, '(abbrev.)': 45}
s_ids = {'': 1, 'mas. fem.': 2, 'fem.': 3, 'super- (pref.).': 4, 'fem. sg.': 5, 'for instects.': 6, 'mas. pl.': 7, 'mas. sg.': 8, 'mas.': 9, 'fem. pl.': 10}

In [226]:
# dont forget last word

def save_word(word, cursor):
    
    # save en_word
    cursor.execute('insert into en_words(en_word) values(?)', (word.en_word,))
    id_en_word = cursor.lastrowid

    
    for def_by_type in word.definitions_by_type:
        word_type = def_by_type.word_type
        id_word_type = t_ids[word_type]
        
        for defi in def_by_type.definitions:
            meaning = defi.definition
            # insert definition
            cursor.execute("insert into definitions(definition, id_en_word, id_word_type) values(?, ?, ?)", (meaning, id_en_word, id_word_type))
            id_definition = cursor.lastrowid
            
            for sub_defi in defi.sub_definitions:
                id_sub_type = s_ids[sub_defi.sub_type]
                
                for am_word_obj in sub_defi.am_words:
                    am_word = am_word_obj.am_word
                    # insert am_word_obj
                    cursor.execute("insert into am_words(am_word, id_en_word, id_definition, id_sub_type) values(?, ?, ?, ?)", (am_word, id_en_word, id_definition, id_sub_type))
                    id_am_word = cursor.lastrowid
                    
                    for example in am_word_obj.examples:
                        # insert examples
                        cursor.execute("insert into examples(example, id_am_word) values(?, ?)", (example, id_am_word))

    
    

        
    

    

In [230]:
# this function reads the 130.000 lines of the file final_content.txt and gets all the words and stores them in the db
def main():
    con = sqlite3.connect("test.sqlite3")
    cursor = con.cursor()

    with open("final_content.txt", "r") as file:
        for line in file:
            if not line:
                break

            if line.startswith('<w>'):
                if content != []:
                    save_word(get_word(content), cursor)
                content = []

            content.append(line)
        
        # last word
        save_word(get_word(content), cursor)
    con.commit()
    con.close()
